### Tools 
##### Models can request to call tools that perform tasks such as fetching data from databases,searching the web, or running code.Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often JSON schema)
2. A function or coroutine to execute

In [4]:
#1. Load dotenv file 
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
#os.environ['GOOGLE_API_KEY']=os.getenv('GOOGLE_API_KEY')
#os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

# As usual we will create a LLM model first 
from langchain.chat_models import init_chat_model
model=init_chat_model(model="gpt-4.1")
response=model.invoke("What is the second longest word in English?")
response

AIMessage(content='The answer to **"What is the second longest word in English?"** depends on what you consider a valid word (dictionary word, coined word, technical term, coined chemical names, etc.). Here’s a breakdown:\n\n### **1. If you count technical or coined words:**\n- The *longest word* is often cited as **pneumonoultramicroscopicsilicovolcanoconiosis** (45 letters)—a lung disease caused by inhaling very fine ash and sand dust.\n- The *second longest* is sometimes listed as **hippopotomonstrosesquipedaliophobia** (36 letters), which ironically means “fear of long words.” However, this is a tongue-in-cheek, pseudo-medical term.\n\n### **2. Real dictionary words (non-coined, non-technical):**\n- **Floccinaucinihilipilification** (29 letters)—the act of estimating something as worthless, is often cited as one of the longest "real" words in major dictionaries.\n- In this context, **antidisestablishmentarianism** (28 letters) is also famous and sometimes listed as the second longe

In [ ]:
from langchain.tools import tool

@tool  # this tool become a decorator and this function will be come a tool for any LLMs
def get_weather(location:str)->str:
    # The purpose of this doc string is - when we bind this tool with LLM, LLM able to identify the functionality of this function 
    """Get the weather at the given location"""
    return f"It's sunny in {location}"

# binding the above tool with LLM/model (this is one way of binding tools with LLM)
model_with_tools=model.bind_tools([get_weather])

# another way binding the tool with LLM is creating an agent (as we did already like below)
agent=create_agent(
    model="gpt-5",
    tools=[get_weather],
    system_prompt="You are an helpful assistant"
)


In [ ]:
# How to call this? Below:
response=model_with_tools.invoke("What's the weather in Dubai?")
print(response)

for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool:{tool_call['name']}")
    print(f"Args:{tool_call['args']}")


content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 51, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-CqjaiqdubbAM2xYAfURrLA7ARgYME', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b5697-be15-7350-b887-cd9710f2316d-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Dubai'}, 'id': 'call_4zWqOLfQHMgduL9bAkz9jbSR', 'type': 'tool_call'}] usage_metadata={'input_tokens': 51, 'output_tokens': 14, 'total_tokens': 65, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Tool:get_weather
Args:{'location': 'Dubai'}


### Tool Execution Loops

In [19]:
# Step 1: Model generates tool calls
messages=[{"role":"user","content":"What's the weather in Dubai?"}]
ai_msg=model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results 
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result=get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass the results back to model for final response
final_response=model_with_tools.invoke(messages)
print(final_response.text)
# The current weather in Dubai is sunny

The weather in Dubai is currently sunny.
